# 9장 - 웹 애플리케이션에 머신 러닝 모델 내장하기

In [25]:
from IPython.display import Image

# 8장 정리 - 영화 리뷰 분류를 위한 모델 훈련

이 절은 8장의 마지막 섹션에서 훈련한 로지스틱 회귀 모델을 다시 사용한다.  
8장에서 만든 `movie_data.csv` 데이터셋을 사용한다.

In [26]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanhyeongu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [28]:
next(stream_docs(path='/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch08/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [29]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [30]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch08/movie_data.csv')

In [31]:
import pyprind
import sys

pbar = pyprind.ProgBar(45, stream=sys.stderr) 

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


In [32]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [33]:
clf = clf.partial_fit(X_test, y_test)

# 학습된 사이킷런 추정기 저장

8장에서 보았듯이 머신 러닝 모델을 훈련하려면 많은 계산 비용이 든다.  
파이썬 인터프리터(interpreter)를 종료하고 나서 새로운 예측을 하거나 웹 애플리케이션을 재시작할 때마다 매번 모델을 다시 훈련하는 것은 바람직하지 않다.  

학습된 모델을 재사용하는 한 가지 방법은 파이썬의 `pickle` 모듈을 사용하는 것이다.  
이 모듈은 파이썬 객체의 구조를 압축된 바이트코드(bytecode)로 직렬화하고 복원할 수 있다. 분류기의 현재 상태를 저장하고 레이블이 없는 새로운 샘플을 분류할 때 훈련 데이터에서 모델을 다시 학습할 필요 없이 저장된 모델을 불러오면 된다. 

In [34]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

이 코드에서 웹 애플리케이션에 필요한 파일과 데이터를 저장할 `movieclassifier` 디렉터리를 만든다. 하드 디스크나 SSD에 있는 `movieclassifier` 디렉터리 안에 `pkl_objects` 서브디렉터리를 만들어 직렬화된 파이썬 객체를 저장한다.  
`pickle` 모듈의 `dump` 메서드를 사용하여 훈련된 로지스틱 회귀 모델뿐만 아니라 `NLTK`(Natural Language ToolKit) 라이브러리의 불용어도 직렬화하여 저장한다. 이렇게 하면 서버에 `NLTK` 라이브러리를 설치할 필요가 없다.  

`dump` 메서드는 첫 번째 매개변수로 대상 객체를 받는다. 두 번째 매개변수로 파이썬 객체가 저장될 파일 객체를 받는다. `pickle` 모듈을 위해 `open` 함수에서 `wb` 매개변수를 사용하여 이진 모드로 파일을 연다. 

`HashingVectorizer`는 학습 과정이 없기 때문에 pickle로 직렬화할 필요가 없다. 그 대신 현재 사용하고 있는 파이썬 세션에서 `HashingVectorizer` 객체를 임포트할 수 있도록 파이썬 스크립트를 만들자.

In [35]:
%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Writing movieclassifier/vectorizer.py


위 코드 셀을 실행한 후 객체가 올바르게 저장되었는지 확인하기 위해 IPython 노트북 커널을 재시작할 수 있다.  

먼저 현재 파이썬 디렉토리를 `movieclassifier`로 변경한다.

In [36]:
import os
os.chdir('movieclassifier')

In [37]:
import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [38]:
import numpy as np
label = {0:'음성', 1:'양성'}

example = ["I love this movie. It's amazing."]
X = vect.transform(example)
print('예측: %s\n확률: %.2f%%' %\
      (label[clf.predict(X)[0]], 
       np.max(clf.predict_proba(X))*100))

예측: 양성
확률: 95.55%


분류기는 정수 클래스 레이블을 예측으로 반환하므로 정수 값을 텍스트 레이블("양성" 또는 "음성")로 매핑하기 위해 간단한 파이썬 딕셔너리를 정의한다. 여기서는 두 개의 클래스를 가진 가잔단한 애플리케이션이지만 딕셔너리 매핑은 다중 클래스 환경에도 적용할 수 있다. 또한, 매핑 딕셔너리는 모델과 함께 저장되어야 한다.   

이 경우 딕셔너리 정의는 한 줄의 코드로 이루어지기 때문에 pickle을 사용해서 직렬화하지 않았다. 하지만 복잡한 매핑 딕셔너리를 가진 실전 애플리케이션에서는 이전 코드 예제처럼 `pickle.dump`와 `pickle.load`를 사용할 수 있다.  

그다음 코드에서 `HashingVectorizer`를 사용하여 샘플 문서를 단어 벡터 `X`로 변환한다. 마지막으로 로지스틱 회귀 모델의 `predict` 메서드를 사용하여 클래스 레이블을 예측하고 `predict_proba` 메서드를 사용하여 예측에 대한 확률을 계산한다. 

# 데이터를 저장하기 위해 SQLite 데이터베이스 설정

이 절에서는 웹 애플리케이션 사용자로부터 피드백을 받아 저장하기 위해 간단한 SQLite 데이터베이스를 셋팅한다.  
SQLite는 오픈 소스 SQL 데이터베이스 엔진이다. 별도의 서버가 필요하지 않아 작은 프로젝트나 간단한 웹 애플리케이션에 적합하다. 기본적으로 SQLite 데이터베이스는 직접 접근할 수 있는 독립된 단일 데이터베이스 파일이라고 생각할 수 있다.   

이 코드를 실행하기 전에 현재 위치가 `moiveclassifier` 디렉토리인지 확인.

In [39]:
os.getcwd()

'/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch09/movieclassifier/movieclassifier'

In [40]:
import sqlite3
import os

'''
먼저 sqlite 라이브러리의 connect 메서드를 호출하여 SQLite 데이터베이스 파일을 연결한다.
movieclassifier 디렉터리에 reviews.sqlite가 없다면 새로운 데이터베이스 파일을 만든다.
'''
conn = sqlite3.connect('reviews.sqlite')


'''
cursor 메서드로 데이터베이스 커서를 만든다. 
커서를 통해 다양한 SQL 문법으로 데이터베이스 레코드를 조작할 수 있다.
'''
c = conn.cursor()



'''
execute 메서드를 실행해 새로운 데이터베이스 테이블 review_db 생성.
review_db 테이블과 함께 review, sentiment, date 세 개의 컬럼도 생성.

물음표(?)를 사용하여 영화 리뷰 텍스트(example1과 example2)와 
클래스 레이블(1과 0)로 이루어진 튜플의 원소를 execute 메서드의 매개변수에 순서대로 전달.
'''
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))


'''
마지막으로 commit  메서드를 사용하여 데이터베이스의 변경 사항을 저장하고 
close 메서드로 데이터베이스와 연결을 닫는다.
'''
conn.commit()
conn.close()

데이터가 테이블에 제대로 저장되었는지 확인하기 위해 데이터베이스에 다시 연결하여 `SELECT` SQL 명령으로 2017년 시작부터 오늘까지 테이블에 추가된 모든 row를 추출

In [41]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2017-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()

In [42]:
print(results)

[('I love this movie', 1, '2022-11-18 09:02:35'), ('I disliked this movie', 0, '2022-11-18 09:02:35')]


# 플라스크 웹 애플리케이션 개발

이전 절에서 영화 리뷰 분류를 위한 코드를 준비했으므로 웹 애플리케이션을 개발하기 위해 플라스크(Flask) 웹 프레임워크의 기본 사항을 알아보자.  

플라스크는 파이썬으로 만들어졌기 때문에 영화 분류기처럼 파이썬 프로그래머가 기존 코드에 편리하게 통합할 수 있는 인터페이스를 제공한다.

## 첫 번째 플라스크 애플리케이션

아주 간단한 웹 애플리케이션을 만들어 보자.  
여기서 만들 첫 번째 애플리케이션은 이름을 입력하는 폼 필드(form field) 하나만 가진 간단한 웹 페이지로 구성된다.  
웹 애플리케이션으로 이름을 입력하면 애플리케이션은 새로운 웹 페이지를 화면에 출력할 것이다.  

먼저 디렉터리 구조를 만든다.  

<pre>
1st_flask_app_1/
    app.py  
    templates/  
      first_app.html    
</pre>

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_01.png?raw=true)

터미널에서 1st_flask_app_1 디렉터리로 이동하여 웹 애플리케이션을 실행하자.
`> python app.py`  
터미널에서 다음 메시지를 볼 수 있다.  

**http://127.0.0.1:5000/**  

위의 주소를 웹 브라우저에 입력하면 실행된 웹 애플리케이션을 볼 수 있다.

## 폼 검증과 화면 출력

플라스크 웹 애플리케이션을 확장해서 HTML 폼 요소를 추가헤보자.  

#### 디렉터리 구조 셋팅
<pre>
1st_flask_app_2/
    app.py
    static/
        style.css
    templates/
        _formhelpers.html
        first_app.html
        hello.html
</pre>

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_02.png?raw=true)

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_03.png?raw=true)

# 영화 리뷰 분류기를 웹 애플리케이션으로 만들기

영화 분류기를 웹 애플리케이션으로 구현해보자.  

사용자가 리뷰를 입력하여 전송하면 새 웹 페이지에서 예측 클래스 레이블과 예측 확률을 볼 수 있다. 또한, 사용자는 맞음 또는 틀림 버튼을 눌러 이 예측 결과에 대한 피드백을 보낼 수 있다.  

사용자가 맞음 또는 틀림 버튼을 누르면 사용자 피드백에 따라 분류 모델을 업데이트할 것이다. 또 버튼을 눌렀을 떄 사용자가 입력한 영화 리뷰 텍스트와 예측한 클래스 레이블을 SQLite 데이터베이스에 저장한다.  

피드백 버튼 중 하나를 눌렀을 때 나타나는 세 번째 웹 페이지는 간단한 감사 메시지와 초기 웹 페이지로 돌아갈 수 있는 리뷰 추가 입력 버튼이 담긴 웹 페이지이다.

## 파일과 폴더: 디렉터리 구조 살펴보기

In [46]:
Image(url='https://git.io/Jts3S', width=200) 

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_04.png?raw=true)

위의 그림과 같이 리뷰를 입력하는 창이 뜨게 된다. 

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_05.png?raw=true)

위와 같이 리뷰를 입력하고 '리뷰 입력' 버튼을 누르면 된다.

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_06.png?raw=true)

리뷰에 대한 예측을 하게 된다. (확률 포함)  

예측이 맞으면 '맞음' 버튼을 틀리면 '틀림'버튼을 클릭하면 된다.

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_07.png?raw=true)

피드백 창이 뜨게 된다.

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_09.png?raw=true)

'생각보다 영화가 재미있지 않았다.'라는 부정적 리뷰는 위 그림과 같이 잘못 분류하는 것을 알 수 있다.  

이럴 때는 '틀림' 버튼을 클릭하고 모델이 추가 학습을 할 수 있게 한다.

![](https://github.com/2hg7274/ML_DL_study/blob/main/ch09/images/09_11.png?raw=true)

'맞음', '틀림' 버튼을 누르게 되면 위 그림과 같이 데이터베이스에 저장되는 것을 알 수 있다.

공개 서버에 웹 애플리케이션 배포는 다음 시간에 이어서...